Start OSM project!

In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import xml.etree.ElementTree as ET  # Use cElementTree or lxml if too slow

OSM_FILE = "austin_texas.osm"  # Replace this with your osm file
SAMPLE_FILE = "austin_sample.osm"

k = 10 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'w') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            print(ET.tostring(element, encoding='iso8859-1'))
            output.write(ET.tostring(element, encoding='iso8859-1'))

    output.write('</osm>')

b'<?xml version=\'1.0\' encoding=\'iso8859-1\'?>\n<node changeset="8497118" id="26546004" lat="30.4695355" lon="-97.7972587" timestamp="2011-06-20T18:36:15Z" uid="388279" user="Tylan" version="15" />\n\t'


TypeError: write() argument must be str, not bytes

In [28]:
# -*- coding: utf-8 -*-

from collections import defaultdict, Counter
import csv
import pprint
import re
import sqlite3
import xml.etree.cElementTree as ET

OSM_FILE = open('austin_texas.osm', 'r', encoding="utf8")

In [15]:
# -*- coding: utf-8 -*-
def get_element(osm_file, tags=('node', 'way', 'relation')):
    context = ET.iterparse(osm_file, events=('start', 'end'))
    __, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()
            
tags = {}
child_tags = {}
count = 0
for element in get_element(OSM_FILE):
    if element.tag in tags.keys():
        tags[element.tag] += 1
    else:
        tags[element.tag] = 1
    for child in element:
        if child.tag in child_tags.keys():
            child_tags[child.tag] += 1
        else:
            child_tags[child.tag] = 1
    
print("\nFor the {}".format(OSM_FILE))
print(tags)
print(child_tags)


For the <_io.TextIOWrapper name='austin_texas.osm' mode='r' encoding='utf8'>
{'relation': 2400, 'node': 6389941, 'way': 669817}
{'tag': 2388234, 'member': 20388, 'nd': 7024440}


In [24]:
# -*- coding: utf-8 -*-
def get_element(osm_file, tags=('node', 'way', 'relation')):
    context = ET.iterparse(osm_file, events=('start', 'end'))
    __, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

popul_elements = []

for element in get_element(OSM_FILE):
    for child in element:
        try:
            if child.attrib['k'] == 'population':
                popul_elements.append(element)
        except:
            try:
                if "Census" in child.attrib['k']:
                    popul_elements.append(element)
            except:
                continue

print("\nFor the {}".format(OSM_FILE))
print(len(popul_elements))




For the <_io.TextIOWrapper name='austin_texas.osm' mode='r' encoding='utf8'>
35


In [29]:
# -*- coding: utf-8 -*-
def get_element(osm_file, tags=('node', 'way', 'relation')):
    context = ET.iterparse(osm_file, events=('start', 'end'))
    __, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

k_tags = {}

for element in get_element(OSM_FILE):
    for child in element:
        try:
            if child.attrib['k'] in k_tags:
                k_tags[child.attrib['k']] += 1
            else:
                k_tags[child.attrib['k']] = 0
        except:
            continue

print("\nFor the {}".format(OSM_FILE))
pprint.pprint(k_tags)




For the <_io.TextIOWrapper name='austin_texas.osm' mode='r' encoding='utf8'>
{'Category': 0,
 'FIXME': 126,
 'FIXME:ref': 1,
 'Grills': 0,
 'ISO3166-1': 0,
 'ISO3166-1:alpha2': 0,
 'ISO3166-1:alpha3': 0,
 'ISO3166-1:numeric': 0,
 'ISO3166-2': 0,
 'Keywords': 0,
 'NHS': 554,
 'Texas_Trunk_System': 143,
 'Type': 1,
 'abandoned': 2,
 'access': 4437,
 'acreage': 0,
 'acres': 0,
 'addr:city': 3709,
 'addr:country': 25,
 'addr:county': 2,
 'addr:door': 1,
 'addr:full': 4,
 'addr:housename': 132,
 'addr:housenumber': 333663,
 'addr:place': 0,
 'addr:postcode': 86641,
 'addr:state': 3352,
 'addr:street': 333621,
 'addr:streetnumber': 0,
 'addr:suite': 0,
 'addr:unit': 25,
 'admin_level': 360,
 'advertising': 2,
 'aeroway': 252,
 'alt_name': 362,
 'alt_name:en': 0,
 'alt_name:vi': 0,
 'amenity': 8152,
 'amperage': 7,
 'area': 487,
 'artist_name': 8,
 'artwork_type': 9,
 'atm': 39,
 'attribution': 8,
 'automated': 1,
 'baby': 0,
 'backrest': 0,
 'barrier': 1224,
 'barrier:personnel': 2,
 'baseb